# 12. Exception Handling

## Exceptions Hierarchy
---

There are **two types of exceptions** in $.NET\,\,Framework$: 
1. `ApplicationException` - exceptions thrown by the *applications* we develop which are defined by application developers and are used by the application software. 
2. `SystemException` - exceptions thrown by the *runtime* which are defined in $.NET$ libraries and are used by the framework.  
   
Each of these is a **base class** within a **hierarchy of `Exception` classes**:

<img src = "_img/ExceptionHeirarchy.jpg" style="display: block; margin: auto;"/>

<br>

### The `Exception` Class

In $.NET\,\,Framework$, `Exception` is the **base class for all exceptions**.    

Several classes inherit directly from it, including `ApplicationException` and `SystemException`.    
   
These two classes are base classes for almost all exceptions that occur during the program execution.   
   
The `Exception` class contains a copy of the call-stack at the time the exception instance was created. The class also has a (usually) short message describing the error (filled in by the method throwing the exception). 

<br>

#### Nested `Exception`s

  
Every `Exception` could have a **nested exception**, also sometimes called an **inner exception**, **wrapped exception**, or **internal exception**.   
   
The ability to wrap an exception with another exception is very useful in some cases and allows exceptions to be linked in the so called **exception chain**.

<br>

In [1]:
// provides the TextReader and StreamReader clasees
using System.IO;

<br>

Let's suppose we have a **software component** (let’s call it **Component** **A**):   

In [2]:
public void ComponentA( string badFileName )
{ 

    // Code that potentially results in an Exception 

    TextReader reader = new StreamReader( badFileName ); 
    string line = reader.ReadLine(); 
    Console.WriteLine(line); 
    reader.Close();

}

<br>

This **Component A**, above,  is then internally used by another **component** (called **Component** **B**). 

In [3]:
public void ComponentB( )
{ 

    // A Bad File Name, with which we intend to test
    // how our program will respond to unexpected inputs:
    string badFileName = "1 will definately%20cause an Exception.pdffff";


    // Add code within the try block that potentially results in an Exception 
    try 
    {
        ComponentA( badFileName );
    }


    // Implement our own handler for the  FileNotFoundException
    // which prints out a meessage explaining the issue and saves the 
    // Stack Trace  in a variable for later reference 
    catch( Exception e )
    {
        Console.WriteLine( 
            $"Oh SNAP! Here's what went wrong in Component B:\n{ e }" 
        );
    }
    
}

<br>

When **B** throws a **B** **exception** (an **exception** defined in **B**),   
**A** will have to *propagate the error*, since it will interfere with any subsequent handler operations.   

In [4]:
ComponentB();

Oh SNAP! Here's what went wrong in Component B:
System.IO.FileNotFoundException: Could not find file 'c:\Users\harry\Dev\projects\Notes On C-Sharp\12. Exception Handling\1 will definately%20cause an Exception.pdffff'.
File name: 'c:\Users\harry\Dev\projects\Notes On C-Sharp\12. Exception Handling\1 will definately%20cause an Exception.pdffff'
   at Microsoft.Win32.SafeHandles.SafeFileHandle.CreateFile(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options)
   at Microsoft.Win32.SafeHandles.SafeFileHandle.Open(String fullPath, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.Strategies.OSFileStreamStrategy..ctor(String path, FileMode mode, FileAccess access, FileShare share, FileOptions options, Int64 preallocationSize, Nullable`1 unixCreateMode)
   at System.IO.Strategies.FileStreamHelpers.ChooseStrategyCore(String path, FileMode mode, FileAccess access, FileShare

Because **B** cannot simply throw an **A**-**exception**,    
it must throw an **B**-**exception** *containing* the **A**-**exception** as a **nested** **exception**.

<br>

### Which Exceptions to Handle and Which Not?

There is *one universal rule* regarding **exception handling**:

> A method should only handle exceptions which it expects and which it knows how to process.<br>All the other exceptions must be left to the calling method.

If we follow this rule and eventually we reach the `Main()` method (or the starting method of the respective thread of execution), if the exception is not yet caught, then the $CLR$ itself will display the error.

A method is considered as *competent to handle an **exception*** if it *expects* this **exception**, and it has the information *why the **exception** has been thrown* and *what to do* in this situation.    
   
If we have a method that must *read a text file* and return its contents as a string, that method might catch `FileNotFoundException` and return an empty string in this case.   
   
Still, this same method will hardly be able to correctly handle `OutOfMemoryException`.   
   
What should the method do in case of insufficient memory? Return an empty string? Throw some other exception? Do something completely different? 
   
So apparently the method is not competent to handle such exception and thus the best way is to pass the exception up to the calling method so it could (hopefully) be handled at some other level by a method competent to do it. 
   
Using this simple philosophy allows exception handling to be done in a structured and systematic way.

<br>

### The `try`-`finally` Construct